In [11]:
import torch
import gpytorch
import botorch
import matplotlib.pyplot as plt
import numpy as np

plt.style.use("bmh")

from tqdm.notebook import tqdm

class Branin:
    """Branin-Hoo f(x1,x2) on [−5,10]×[0,15]"""
    def __init__(self):
        self.bounds = torch.tensor([[-5.0, 0.0], [10.0, 15.0]])
        # recommended constants
        self.a, self.b = 1.0, 5.1/(4*torch.pi**2)
        self.c, self.r = 5.0/torch.pi, 6.0
        self.s, self.t = 10.0, 1.0/(8*torch.pi)
    def __call__(self, x: torch.Tensor) -> torch.Tensor:
        x1, x2 = x[...,0], x[...,1]
        term1 = self.a*(x2 - self.b*x1**2 + self.c*x1 - self.r)**2
        term2 = self.s*(1 - self.t)*torch.cos(x1) + self.s
        return (term1 + term2).squeeze(-1)

def visualize_gp_belief_and_policy(model, likelihood, policy=None, next_x=None):
    # Disabled for 2D Branin function; kept for compatibility
    pass

class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

def fit_gp_model(train_x, train_y, num_train_iters=500):
    # Declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # Train the hyperparameters
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in tqdm(range(num_train_iters), desc="Training GP"):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

# Initialize Branin function and bounds
branin = Branin()
bounds = branin.bounds  # [[-5, 0], [10, 15]]

# Parameters
n_initial_points = 10
num_queries = 20
num_runs = 10

# Store minimum f(x) for each run
all_min_f_values = []

for run in range(num_runs):
    print(f"\nRun {run + 1}/{num_runs}")
    
    # Set different random seed for each run
    torch.manual_seed(2 + run)
    
    # Initialize with 10 random points in [-5, 10] x [0, 15]
    train_x = torch.rand(size=(n_initial_points, 2))
    train_x = bounds[0] + (bounds[1] - bounds[0]) * train_x  # Scale to bounds
    train_y = branin(train_x)

    # Print initial points and their function values
    print("Initial points (x1, x2, f(x)):")
    print(torch.hstack([train_x, train_y.unsqueeze(1)]))

    # Fit initial GP model
    model, likelihood = fit_gp_model(train_x, train_y)

    # Track minimum f(x) for this run
    min_f_values = [train_y.min().item()]

    # Run Bayesian optimization
    for i in range(num_queries):
        print(f"Iteration {i}, incumbent x=({train_x[train_y.argmin()][0].item():.4f}, {train_x[train_y.argmin()][1].item():.4f}), f(x)={train_y.min().item():.4f}")

        model, likelihood = fit_gp_model(train_x, train_y)

        policy = botorch.acquisition.analytic.ProbabilityOfImprovement(
            model, best_f=train_y.min(), maximize=False
        )
        
        next_x, acq_val = botorch.optim.optimize_acqf(
            policy,
            bounds=bounds,
            q=1,
            num_restarts=20,
            raw_samples=50,
        )

        next_y = branin(next_x)

        # Ensure next_y is 1D with shape (1,)
        next_y = next_y.unsqueeze(0) if next_y.dim() == 0 else next_y

        train_x = torch.cat([train_x, next_x])
        train_y = torch.cat([train_y, next_y])
        
        # Store the minimum f(x)
        min_f_values.append(train_y.min().item())

    # Store results for this run
    all_min_f_values.append(min_f_values)

    # Plot minimum f(x) vs iteration for this run
    plt.figure(figsize=(8, 6))
    plt.plot(range(num_queries + 1), min_f_values, marker='o', linestyle='-', color='b')
    plt.xlabel('Iteration')
    plt.ylabel('Minimum f(x)')
    plt.title(f'Minimum f(x) vs Iteration (Run {run + 1})')
    plt.grid(True)
    plt.savefig(f'min_f_vs_iteration_run_{run + 1}.png')
    plt.close()

# Convert to numpy array for easier computation
all_min_f_values = np.array(all_min_f_values)  # Shape: (num_runs, num_queries + 1)

# Compute mean, std, and 95% CI
mean_f = np.mean(all_min_f_values, axis=0)
std_f = np.std(all_min_f_values, axis=0)
ci_95 = 1.96 * std_f / np.sqrt(num_runs)  # 95% CI: z=1.96 for normal distribution

# Plot convergence with mean, std, and CI
plt.figure(figsize=(10, 6))
iterations = range(num_queries + 1)

# Plot mean
plt.plot(iterations, mean_f, marker='o', linestyle='-', color='b', label='Mean Minimum CLEAR f(x)')

# Plot 95% CI
plt.fill_between(
    iterations,
    mean_f - ci_95,
    mean_f + ci_95,
    color='b',
    alpha=0.2,
    label='95% Confidence Interval'
)

# Plot mean ± std
plt.fill_between(
    iterations,
    mean_f - std_f,
    mean_f + std_f,
    color='g',
    alpha=0.1,
    label='Mean ± Std'
)

plt.xlabel('Iteration')
plt.ylabel('Minimum f(x)')
plt.title('Convergence Plot: Mean, Std, and 95% CI over 10 Runs (Branin)')
plt.grid(True)
plt.legend()
plt.savefig('convergence_plot.png')
plt.close()


Run 1/10
Initial points (x1, x2, f(x)):
tensor([[ 4.2204,  5.7152, 22.5310],
        [ 4.5567,  7.1169, 40.8511],
        [ 5.7039,  9.2855, 84.6316],
        [ 1.6380,  1.4365, 14.6597],
        [ 4.2124,  0.8599,  5.9265],
        [ 3.4857,  7.9984, 36.6794],
        [ 0.8508, 13.6327, 95.4217],
        [ 3.0005, 10.6101, 68.1029],
        [ 5.6740,  3.0755, 21.6653],
        [-0.3833, 14.7129, 84.2555]])


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 0, incumbent x=(4.2124, 0.8599), f(x)=5.9265


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 1, incumbent x=(4.2124, 0.8599), f(x)=5.9265


...

Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 18, incumbent x=(9.4472, 2.5884), f(x)=0.4092


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]

Iteration 19, incumbent x=(9.4468, 2.5813), f(x)=0.4079


Training GP:   0%|          | 0/500 [00:00<?, ?it/s]